In [11]:
from playground.save_activations import layer_signature
%load_ext autoreload
%autoreload 2

from amber.adapters.text_snippet_dataset import TextSnippetDataset
from amber.core.language_model import LanguageModel
from pathlib import Path

from playground.train_sae import STORE_DIR

MODEL_ID = "sshleifer/tiny-gpt2"  # tiny model for quick experimentation
HF_DATASET = "roneneldan/TinyStories"

CACHE_DIR = Path("./store/tinystories")
STORE_DIR = Path("./store/tiny-gpt2")
DEVICE = 'cpu'
DATA_SPLIT = "train"
TEXT_FIELD = "text"
DATA_LIMIT = 200  # keep small for a quick demo
MAX_LENGTH = 128

model = LanguageModel.from_huggingface(MODEL_ID)
model.model.to(DEVICE)

dataset = TextSnippetDataset.from_huggingface(
    HF_DATASET,
    split=DATA_SPLIT,
    cache_dir=str(CACHE_DIR),
    text_field=TEXT_FIELD,
    limit=DATA_LIMIT,
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Saving the dataset (1/1 shards): 100%|██████████| 200/200 [00:00<00:00, 104661.36 examples/s]


In [16]:
import os
from amber.mechanistic.autoencoder.autoencoder import Autoencoder

sae_models = STORE_DIR / "sae_models"
last_run = os.listdir(sae_models)[-1]

sae_layer = Autoencoder.load_model(sae_models / last_run / 'final.pt')[0]
sae_layer.metadata

2025-09-30 20:33:06,258 [INFO] amber.mechanistic.autoencoder.autoencoder: 
Loaded model from store/tiny-gpt2/sae_models/tinystories_20250930_203105/final.pt
n_latents=2, n_inputs=2, activation=TopK_4, tied=False


{'run_name': 'tinystories_20250930_203105',
 'model': 'GPT2LMHeadModel',
 'layer_signature': 'gpt2lmheadmodel_transformer_h_1_ln_2',
 'dataset': {'cache_dir': 'store/tinystories', 'length': 200},
 'options': {'dtype': None,
  'max_length': 128,
  'save_inputs': True,
  'batch_size': 8}}

In [17]:
model_name = sae_layer.metadata['model']
layer_signature = sae_layer.metadata['layer_signature']


model = LanguageModel.from_huggingface(
    "sshleifer/tiny-gpt2",
)

In [19]:
model.layers.register_new_layer('sae', sae_layer, layer_signature)